In [1]:
using Revise
using Bilevel

using ForwardDiff
using DiffResults
using LinearAlgebra
using Plots
using BenchmarkTools

┌ Info: Precompiling Bilevel [9688c538-179f-11e9-3174-495cea6b7f67]
└ @ Base loading.jl:1186
┌ Error: evaluation error
│   mod = Bilevel
│   ex = const usrfun = #= /Users/blandry/.julia/dev/Bilevel/src/solvers/snopt.jl:154 =# @cfunction(objcon_wrapper, Cvoid, (Ptr{Clong}, Ref{Clong}, Ptr{Cdouble}, Ref{Clong}, Ref{Clong}, Ptr{Cdouble}, Ref{Clong}, Ref{Clong}, Ptr{Cdouble}, Ptr{Cchar}, Ref{Clong}, Ptr{Clong}, Ref{Clong}, Ptr{Cdouble}, Ref{Clong}))
│   exception = (ErrorException("invalid lookup expr (\$(QuoteNode(Core.svec)))(JuliaInterpreter.SSAValue(2), JuliaInterpreter.SSAValue(3), JuliaInterpreter.SSAValue(4), JuliaInterpreter.SSAValue(5), JuliaInterpreter.SSAValue(6), JuliaInterpreter.SSAValue(7), JuliaInterpreter.SSAValue(8), JuliaInterpreter.SSAValue(9), JuliaInterpreter.SSAValue(10), JuliaInterpreter.SSAValue(11), JuliaInterpreter.SSAValue(12), JuliaInterpreter.SSAValue(13), JuliaInterpreter.SSAValue(14), JuliaInterpreter.SSAValue(15), JuliaInterpreter.SSAValue(16))"), Union{Ptr{

# Nonlinear problems

In [ ]:
function get_rosenbrock()
    function eval_obj(x::AbstractArray{T}) where T
        f = (1 - x[1])^2 + 100*(x[2] - x[1]^2)^2

        f
    end
    
    function eval_cons(x::AbstractArray{T}) where T
        g = vcat(-5. .- x, x .- 5.)
        
        g
    end
    
    return Bilevel.generate_autodiff_solver_fn(eval_obj,eval_cons,[],1:4)
end

solver_fn = get_rosenbrock()

x0 = [3.0; 3.0]
options = Dict{String, Any}()
options["Derivative option"] = 1
options["Verify level"] = 1
xopt, info = Bilevel.snopt(solver_fn, 0, 4, x0, options)

display(info)
display(xopt)

x0 = [3.0; 3.0]
options = Dict{String, Any}()
options["num_fosteps"] = 1
options["num_sosteps"] = 5
options["c"] = 1.
options["c_fos"] = 1.
options["c_sos"] = 1.
xopt, info = Bilevel.auglag(solver_fn, 0, 4, x0, options)

display(info)
display(xopt)

In [ ]:
function get_barnesgrad()
    a1 = 75.196
    a3 = 0.12694
    a5 = 1.0345e-5
    a7 = 0.030234
    a9 = 3.5256e-5
    a11 = 0.25645
    a13 = 1.3514e-5
    a15 = -5.2375e-6
    a17 = 7.0e-10
    a19 = -1.6638e-6
    a21 = 0.0005
    a2 = -3.8112
    a4 = -2.0567e-3
    a6 = -6.8306
    a8 = -1.28134e-3
    a10 = -2.266e-7
    a12 = -3.4604e-3
    a14 = -28.106
    a16 = -6.3e-8
    a18 = 3.4054e-4
    a20 = -2.8673
    
    function eval_obj(x::AbstractArray{T}) where T
        x1 = x[1]
        x2 = x[2]
        y1 = x1*x2
        y2 = y1*x1
        y3 = x2^2
        y4 = x1^2

        f = a1 + a2*x1 + a3*y4 + a4*y4*x1 + a5*y4^2 +
            a6*x2 + a7*y1 + a8*x1*y1 + a9*y1*y4 + a10*y2*y4 +
            a11*y3 + a12*x2*y3 + a13*y3^2 + a14/(x2+1) +
            a15*y3*y4 + a16*y1*y4*x2 + a17*y1*y3*y4 + a18*x1*y3 +
            a19*y1*y3 + a20*exp(a21*y1)
        
        f
    end

    function eval_cons(x::AbstractArray{T}) where T
        x1 = x[1]
        x2 = x[2]
        y1 = x1*x2
        y2 = y1*x1
        y3 = x2^2
        y4 = x1^2
        
        g = zeros(T, 3)
        g[1] = 1 - y1/700.0
        g[2] = y4/25.0^2 - x2/5.0
        g[3] = (x1/500.0- 0.11) - (x2/50.0-1)^2
        
        g
    end
    
    fres = DiffResults.HessianResult(zeros(2))
    fcfg = ForwardDiff.HessianConfig(eval_obj, fres, zeros(2))
    gres = DiffResults.JacobianResult(zeros(3), zeros(2))
    gcfg = ForwardDiff.JacobianConfig(eval_cons, zeros(2))
    
    return Bilevel.generate_autodiff_solver_fn(eval_obj,fres,fcfg,eval_cons,gres,gcfg,[],1:3)
end

solver_fn = get_barnesgrad()

x0 = [10., 10.]
options = Dict{String, Any}()
options["Derivative option"] = 1
options["Verify level"] = 1
xopt, info = Bilevel.snopt(solver_fn, 0, 3, x0, options)

display(info)
display(xopt)

x0 = [10., 10.]
options = Dict{String, Any}()
options["num_fosteps"] = 5
options["num_sosteps"] = 15
options["c"] = 1.
options["c_fos"] = 10.
options["c_sos"] = 1.
xopt, info = Bilevel.auglag(solver_fn, 0, 3, x0, options)

display(info)
display(xopt)

# Bilevel problem

In [2]:
# Problem taken from:
# Sinha, Ankur, Pekka Malo, and Kalyanmoy Deb. 
# "A review on bilevel optimization: from classical to evolutionary approaches and applications." 
# IEEE Transactions on Evolutionary Computation 22.2 (2018): 276-295.

α = 10.
β = .2

δl = 1.
δf = 2.
γl = .3
γf = 5.8
cl = 10.
cf = 1.

P = (ql,qf) -> α - β*(ql + qf)
Cl = ql -> δl*ql*ql + γl*ql + cl
Cf = qf -> δf*qf*qf + γf*qf + cf

#7 (generic function with 1 method)

In [3]:
# closed form solution

qlopt = (2. * (β + δf)*(α - γl)-β * (α - γf))/(4. * (β + δf)*(β + δl) - 2. * β^2)
qfopt = (α - γf)/(2. * (β + δf)) - (β * (α - γl) - (β^2 * (α - γf))/(2. * (β + δf)))/(4. * (β + δf) * (β + δl) - 2. * β^2)

display(qlopt)
display(qfopt)

3.99236641221374

0.7730742539902844

In [17]:
# solution using our bilevel solver

f_options = Dict{String, Any}()
f_options["num_fosteps"] = 1
f_options["num_sosteps"] = 10
f_options["c"] = 1.
f_options["c_fos"] = 1.
f_options["c_sos"] = 1.

function solve_follower(ql::AbstractArray{U}) where U
    function eval_obj_(qf::AbstractArray{T}) where T
        f = -(P(ql[1],qf[1])*qf[1] - Cf(qf[1]))
        
        f
    end
    
    function eval_cons_(qf::AbstractArray{T}) where T
        g = -qf
        
        g
    end
    
    fres = DiffResults.HessianResult(zeros(U, 1))
    gres = DiffResults.JacobianResult(zeros(U, 1), zeros(U, 1))
    solver_fn = Bilevel.generate_autodiff_solver_fn(eval_obj_,fres,eval_cons_,gres,[],1:1)
    
    qf0 = zeros(1)

    qfopt, info = Bilevel.auglag(solver_fn, 0, 1, qf0, f_options)

    qfopt
end

function get_bilevel()
    function eval_obj(ql::AbstractArray{T}) where T
        qfopt = solve_follower(ql)
        
        f = -(P(ql[1],qfopt[1])*ql[1] - Cl(ql[1]))
        
        f
    end

    function eval_cons(ql::AbstractArray{T}) where T        
        g = -ql
        
        g
    end
    
    return Bilevel.generate_autodiff_solver_fn(eval_obj,eval_cons,[],1:1,hessian=false)
end

solver_fn = get_bilevel()

x0 = [0.]
options = Dict{String, Any}()
options["Derivative option"] = 1
options["Verify level"] = -1
options["Major optimality tolerance"] = 1e-3

qlopt, info = Bilevel.snopt(solver_fn, 0, 1, x0, options)

qfopt = solve_follower(qlopt)

display(info)
display(qlopt)
display(qfopt)

"Finished successfully: optimality conditions satisfied"

1-element Array{Float64,1}:
 3.992366412213758

1-element Array{Float64,1}:
 0.7730742539903092